In [1]:
# import rasterio
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
# from shapely.geometry import Polygon, box
# import geopandas as gpd
import ee
import geemap
# from rasterio import features
import json
import os
import folium
# ee.Authenticate()
ee.Initialize()


In [2]:
from tools import filteringModule

In [3]:
help(ee.Filter)

Help on class Filter in module ee.filter:

class Filter(ee.computedobject.ComputedObject)
 |  Filter(*args, **kwargs)
 |  
 |  An object to represent collection filters.
 |  
 |  Method resolution order:
 |      Filter
 |      ee.computedobject.ComputedObject
 |      ee.encodable.Encodable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Not(self)
 |      Returns the opposite of this filter.
 |      
 |      Returns:
 |        The negated filter, which will match iff this filter doesn't.
 |  
 |  __init__(self, opt_filter=None)
 |      Construct a filter.
 |      
 |      This constructor accepts the following args:
 |        1) Another filter.
 |        2) An array of filters (which are implicitly ANDed together).
 |        3) A ComputedObject returning a filter. Users shouldn't be making these;
 |           they're produced by the generator functions below.
 |      
 |      Args:
 |        opt_filter: Optional filter to add.
 |  
 |  predicateCount(self)
 |      Return

In [92]:
# folder = (new Function('return __dirname'))()
# filteringModule = require('filters.js')
city_name = 'Bangalore Urban'
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
selected = admin2.filter(ee.Filter.eq('ADM2_NAME', city_name))
geometry = selected.geometry()
geom = geometry.bounds()
# print('Bangalore Bounds:', geom)

year_start = '2020'
year_end = '2023'
image_before_start = '2015-11-01'
image_before_end = '2016-03-01'
image_after_start = '2023-01-01'
image_after_end = '2023-04-01'
DW_bands = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
image_col_ = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1"). \
            filterDate(image_before_start, image_after_end) \
            .filterBounds(geom) \

mossaic = filteringModule.mosaicSamedayImage(image_col_)      
filled_col = filteringModule.forward_fill(mossaic)      
monthlyMedians = filteringModule.computeMonthlyMedians(filled_col)

# print(monthlyMedians)
# print(ee.Image(monthlyMedians.toList(monthlyMedians.size()).get(30)).bandNames().size())
monthlyMedians = monthlyMedians.map(filteringModule.addBandnumber).filter(ee.Filter.gt('numBands',0))

band = 'flooded_vegetation'
band_name = 'label'
dw =  monthlyMedians.select(band_name)
beforeDw = dw.filterDate(image_before_start, image_before_end).mean()
afterDw = dw.filterDate(image_after_start, image_after_end).mean()
newUrban = beforeDw.lt(0.3).And(afterDw.gt(0.7))
lon, lat = geometry.centroid().getInfo()['coordinates']
# Map = folium.Map(location= [lat, lon], zoom_start=12)

Map = geemap.Map()
changeVisParams = {'min': 0, 'max': 1, 'palette': ['white', 'red']}
# Map.addLayer(dw.first().mask().clip(geometry), changeVisParams, 'Urban')
# Map.addLayer(dw.first().mask().Not(), {}, 'Urban')
Map.addLayer(newUrban, changeVisParams, "New Urban")
# Map.addLayer(monthlyMedians.first().select(['built', 'trees', 'water']).clip(geom), {}, 'Bangalore')
Map.centerObject(geometry, 12)
Map

Map(center=[12.942208526432214, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…

In [93]:
# # Function to calculate the statistics for an image
# def calculateStatistics(image):
#     statistics = image.gt(0.7).reduceRegion(
#         reducer=ee.Reducer.sum(),  # Use the desired reducer function, such as mean, sum, max, etc.
#         geometry=geometry,
#         scale=20  # Set the appropriate scale based on the image resolution
#     )
#     return image.set(statistics)

# # Apply the calculateStatistics function over the image collection
# image_collection_with_stats = dw.map(calculateStatistics)

# # Get the resulting image collection with statistics
# image_collection_with_stats

In [94]:
img1 = ee.Image(mossaic.toList(mossaic.size()).get(28))
img2 = ee.Image(filled_col.toList(filled_col.size()).get(28))
img1

In [95]:
reduced = dw.map(lambda image: ee.Feature(None, image.eq(DW_bands.index(band)).reduceRegion(reducer = ee.Reducer.sum(), geometry= geom, scale = 20)\
                                          .set('month', image.get('month'))\
                                           .set('year', image.get('year')) ))
area_collection_dict = reduced.getInfo()

In [96]:
feature_list = area_collection_dict.get('features')
feature_count = len(feature_list)
areas = (np.array([i.get('properties').get(band_name) for i in feature_list])*20)**2
month_data = np.array([i.get('properties').get('month') for i in feature_list])
year_data = np.array([i.get('properties').get('year') for i in feature_list])
areas

array([1.50543635e+11, 1.57672526e+11, 2.70962055e+11, 1.65924406e+11,
       1.54198014e+11, 1.54198014e+11, 1.54198014e+11, 2.34365644e+11,
       9.14002278e+10, 9.34431394e+10, 5.16471076e+10, 9.21972496e+10,
       1.27460536e+11, 1.58078850e+10, 1.07133703e+10, 1.56934473e+10,
       1.56934473e+10, 1.57385782e+10, 8.05308601e+11, 1.08293161e+12,
       1.13685486e+12, 7.94570050e+11, 5.19704613e+11, 3.50289625e+11,
       1.99582444e+11, 2.73419243e+11, 2.79340354e+11, 2.89058541e+11,
       3.14548280e+11, 1.99492054e+11, 1.34122546e+11, 8.67644847e+10,
       5.48436887e+10, 3.87417247e+10, 2.40560100e+10, 2.43547236e+10,
       2.44798789e+10, 2.44204722e+10, 2.44204722e+10, 2.43177239e+10,
       2.45835591e+10, 1.81496181e+11, 1.85825842e+11, 2.20806010e+11,
       2.58322806e+11, 1.39508929e+11, 1.57136504e+11, 1.72246939e+11,
       1.39882160e+11, 1.61137513e+11, 4.38481210e+11, 2.40970490e+11,
       2.03955311e+11, 3.05212052e+11, 2.75004738e+11, 1.96446819e+11,
      

In [97]:
dir_path = f"app/data/{city_name}/{band}"
os.makedirs(dir_path, exist_ok=True)
with open(f"{dir_path}/areas.json", 'w') as dest:
    json.dump(feature_list, dest, indent = 6)

In [98]:
unique_year = np.sort(np.unique(year_data))[1:]
months_box = [True, True, True, True, True, True, True, False, False, False, False, False]
yearly_data = [np.mean(areas[year_data == i][0:6]) for i in unique_year]
yearly_data

[176192171337.69058,
 52253266009.03858,
 545736805591.9191,
 42206751760.78431,
 181317224739.2008,
 188802194295.594,
 1256279136164.6462,
 1930323358004.7942]

In [99]:
import pandas as pd
import seaborn as sns

In [100]:
df = pd.DataFrame({"year": year_data, "month": month_data, "areas": areas})
df['date'] = pd.to_datetime(dict(year=df.year, month=df.month, day = 1))
df.set_index('date', inplace=True)
df = df.resample('MS').asfreq()
df.fillna(method='ffill', inplace=True)
df.month = df.index.month
df.year = df.index.year
df

,year,month,areas
date,,,
2015-12-01,2015,12,1.505436e+11
2016-01-01,2016,1,1.576725e+11
2016-02-01,2016,2,2.709621e+11
2016-03-01,2016,3,2.709621e+11
2016-04-01,2016,4,1.659244e+11
...,...,...,...
2022-11-01,2022,11,1.985920e+12
2022-12-01,2022,12,1.360106e+12
2023-01-01,2023,1,1.508107e+12


In [101]:
import plotly.express as px

In [102]:
px.line(x=unique_year, y=yearly_data)

In [103]:
df_filtered = df[df.month.isin([1,2,3, 4])]
px.line(df, x='month', y='areas', color = 'year')